In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
#로컬 모듈 import
import sys
import os
print(os.getcwd())
# 모듈 경로를 sys.path에 추가
module_path = os.path.abspath(os.path.join('..', 'my_transformer'))
if module_path not in sys.path:
    sys.path.append(module_path)
from my_transformer.my_transformer import Transformer

c:\Users\박동연\Documents\GitHub\24-2_DS_assignment\Week_01\Transformer


In [7]:
from torch.utils.data import Dataset, DataLoader
class PatternDataset(Dataset):
    def __init__(self, num_samples=1000, sequence_length=4, max_num=10):
        self.data = []
        self.targets = []
        for _ in range(num_samples):
            start = torch.randint(0, max_num, (1,)).item()  # 시작 숫자
            diff = torch.randint(1, 5, (1,)).item()  # 등차
            sequence = [start + i * diff for i in range(sequence_length)]
            next_value = sequence[-1] + diff  # 다음에 올 숫자
            
            # 입력 시퀀스는 정수로 이루어진 시퀀스, 타겟은 다음에 올 숫자
            self.data.append(torch.tensor(sequence))
            self.targets.append(next_value)
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

# 하이퍼파라미터 설정
src_vocab_size = 100
tgt_vocab_size = 200
d_model = 32
n_heads = 4
d_ff = 64
num_encoder_layers = 6
num_decoder_layers = 6
dropout = 0.1
batch_size = 32
num_epochs = 50
learning_rate = 0.0001

# 데이터셋 및 데이터로더 생성
dataset = PatternDataset(num_samples=1000)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 모델 초기화
model = Transformer(src_vocab_size, tgt_vocab_size, d_model, n_heads, d_ff, num_encoder_layers, num_decoder_layers, dropout)

# 손실 함수 및 최적화 도구 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 학습 루프
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for src, tgt in dataloader:
        # 입력 시퀀스 준비
        tgt_input = torch.zeros_like(tgt).unsqueeze(1)  # 예측 시작을 위한 빈 타겟 시퀀스
        tgt = tgt.unsqueeze(1)  # 타겟을 2D 텐서로 변환
        
        optimizer.zero_grad()

        # 모델에 입력 및 타겟 시퀀스 전달
        output = model(src, tgt_input)

        # 출력 크기 조정 및 손실 계산
        output = output.view(-1, tgt_vocab_size)  # (batch_size, vocab_size)
        tgt = tgt.view(-1)  # (batch_size)
        
        loss = criterion(output, tgt)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader):.4f}")

Epoch 1/50, Loss: 4.8806
Epoch 2/50, Loss: 4.5984
Epoch 3/50, Loss: 4.4451
Epoch 4/50, Loss: 4.3399
Epoch 5/50, Loss: 4.2288
Epoch 6/50, Loss: 4.1191
Epoch 7/50, Loss: 4.0210
Epoch 8/50, Loss: 3.9173
Epoch 9/50, Loss: 3.8247
Epoch 10/50, Loss: 3.7476
Epoch 11/50, Loss: 3.6705
Epoch 12/50, Loss: 3.5892
Epoch 13/50, Loss: 3.5289
Epoch 14/50, Loss: 3.4829
Epoch 15/50, Loss: 3.4333
Epoch 16/50, Loss: 3.3886
Epoch 17/50, Loss: 3.3526
Epoch 18/50, Loss: 3.3253
Epoch 19/50, Loss: 3.2969
Epoch 20/50, Loss: 3.2682
Epoch 21/50, Loss: 3.2447
Epoch 22/50, Loss: 3.2261
Epoch 23/50, Loss: 3.2051
Epoch 24/50, Loss: 3.1976
Epoch 25/50, Loss: 3.1838
Epoch 26/50, Loss: 3.1701
Epoch 27/50, Loss: 3.1608
Epoch 28/50, Loss: 3.1513
Epoch 29/50, Loss: 3.1386
Epoch 30/50, Loss: 3.1283
Epoch 31/50, Loss: 3.1296
Epoch 32/50, Loss: 3.1127
Epoch 33/50, Loss: 3.1146
Epoch 34/50, Loss: 3.1107
Epoch 35/50, Loss: 3.1125
Epoch 36/50, Loss: 3.0981
Epoch 37/50, Loss: 3.0895
Epoch 38/50, Loss: 3.0918
Epoch 39/50, Loss: 3.

In [5]:
# 테스트 데이터셋 생성 (학습 데이터와 동일한 방식으로 생성)
test_dataset = PatternDataset(num_samples=1000)  # 테스트용 샘플 수
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

def test_model(model, dataloader):
    model.eval()  # 평가 모드로 설정
    total_correct = 0
    total_samples = 0

    with torch.no_grad():  # 그래디언트 계산 중지 (평가 시에는 필요하지 않음)
        for src, tgt in dataloader:
            # 입력 시퀀스 준비
            tgt_input = torch.zeros_like(tgt).unsqueeze(1)  # 예측 시작을 위한 빈 타겟 시퀀스
            tgt = tgt.unsqueeze(1)  # 타겟을 2D 텐서로 변환
            
            # 모델에 입력 시퀀스를 전달하고 예측 값 생성
            output = model(src, tgt_input)
            predicted = output.argmax(dim=-1)  # 예측 결과는 argmax를 통해 얻음
            
            # 실제 타겟과 예측값 비교
            correct = (predicted.view(-1) == tgt.view(-1)).sum().item()
            total_correct += correct
            total_samples += tgt.size(0)

    # 정확도 계산
    accuracy = total_correct / total_samples * 100
    print(f"Test Accuracy: {accuracy:.2f}%")

# 학습된 모델 테스트
test_model(model, test_dataloader)


Test Accuracy: 11.00%
